# RO change over time

## Imports

In [ ]:
import warnings
import datetime
import matplotlib
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import seaborn as sns
import xarray as xr
from climpred import HindcastEnsemble
from dateutil.relativedelta import *
from matplotlib.ticker import AutoMinorLocator
import warnings
import tqdm
import pathlib
import cmocean

# Import custom modules
from src.XRO import XRO, xcorr

## set plotting specs
sns.set(rc={"axes.facecolor": "white", "axes.grid": False})

## bump up DPI
mpl.rcParams["figure.dpi"] = 100

## Load data

In [ ]:
## MPI data
mpi_load_fp = pathlib.Path("/Users/theo/research/enso2025_xro/data/mpi_Th/Th.nc")
Th = xr.open_dataset(mpi_load_fp)

## ORAS5 reanalysis (use as benchmark)
oras_load_fp = pathlib.Path("../data/XRO_indices_oras5.nc")
Th_oras = xr.open_dataset(oras_load_fp)

## rename variables for consistency
Th_oras = Th_oras[["Nino34", "WWV"]].rename({"Nino34": "T_34", "WWV": "h"})

## Evaluate ENSO in MPI

Get subset of MPI which overlaps with RO

In [ ]:
Th_hist = Th.sel(time=slice("1979", "2024"))

### Seasonal Synchronization

In [ ]:
## func to compute std dev as a function of month
get_std = lambda x: x.groupby("time.month").std("time")

## compute std for each dataset
oras_std = get_std(Th_oras["T_34"])
mpi_std_mean = get_std(Th_hist["T_34"]).mean("member")
mpi_std_std = get_std(Th_hist["T_34"]).std("member")

## months (x-coordinate for plotting
months = np.arange(1, 13)

### Set up plot
fig, ax = plt.subplots(figsize=(4, 3))

## plot for ORAS5
oras_plot = ax.plot(months, oras_std, label="ORAS5")

## plot MPI ensemble mean
mpi_plot = ax.plot(months, mpi_std_mean, label="MPI")

## plot ± bounds for MPI
kwargs = dict(c=mpi_plot[0].get_color(), ls="--", lw=1)
for s in [1, -1]:
    ax.plot(months, s * mpi_std_std + mpi_std_mean, **kwargs)

## adjust limits and label
ax.set_ylim([0, None])
ax.set_yticks([0.5, 1])
ax.set_xticks([1, 5, 12], labels=["Jan", "May", "Dec"])
ax.set_xlabel("Month")
ax.set_ylabel(f"$\\sigma(T)$")
ax.set_title("Seasonal synchronization")
ax.legend()
plt.show()

### Power spectrum

### $T$, $h$ cross-correlation

## Check: can RO reproduce them?

Function to fit RO to ensemble

In [ ]:
def get_ensemble_params(data, T_var="T_3", h_var="h_w", verbose=False):
    """get RO params for each ensemble member"""

    ## empty list to hold params
    params = []

    ## Loop thru ensemble members
    for m in tqdm.tqdm(data.member, disable=not (verbose)):

        ## initialize model
        model = XRO(ncycle=12, ac_order=1, is_forward=True)

        ## select ensemble member and variables
        data_subset = data[[T_var, h_var]].sel(member=m)

        ## fit model
        with warnings.catch_warnings(action="ignore"):
            fit = model.fit_matrix(data_subset, maskNT=[], maskNH=[])

        ## append to list of parameters
        params.append(model.get_RO_parameters(fit))

    return xr.concat(params, dim=data.member)

## Look at parameter change over time

Next, define a function to compute parameter changes over time

In [ ]:
def get_ensemble_params_over_time(
    data, T_var="T_3", h_var="h_w", window_size=360, step_size=60
):
    """Get RO params for each ensemble member as a function of time.
    Args:
        window_size: size of sliding window (units: months)
        step_size: how many months to slide the window between each calculation
    """

    ## Get number of timesteps in data
    n = len(data.time)

    ## empty list to hold results and dates
    params_by_year = []
    start_dates = []

    ## loop through rolling windows
    for i in tqdm.tqdm(np.arange(0, n - step_size, step_size)):

        ## make sure there's enough samples for robust estimate
        if (n - i) > (0.7 * window_size):

            ## get subset of data for fitting model
            data_subset = data.isel(time=slice(i, i + window_size))

            ## Get start date for subset
            start_dates.append(data_subset.time.isel(time=0))

            ## get parameter fit
            params_by_year.append(
                get_ensemble_params(
                    data_subset, T_var=T_var, h_var=h_var, verbose=False
                )
            )

    ## convert from list to xarray
    start_dates = xr.concat(start_dates, dim="time")
    params_by_year = xr.concat(params_by_year, dim=start_dates)

    return params_by_year

### Do the computation and save parameters to file

In [ ]:
## specify save filepath
save_fp = pathlib.Path("/Users/theo/research/enso2025_xro/results/params.nc")

## Load parameters if already computed
if save_fp.is_file():
    params = xr.open_dataset(save_fp)

else:
    ## compute params
    params = get_ensemble_params_over_time(Th, window_size=360, step_size=60)

    ## save to file
    params.to_netcdf(save_fp)

### Plot diagnostics

#### Ensemble-mean growth rate as a function of seasonal cycle and time

In [ ]:
fig, ax = plt.subplots(figsize=(3.5, 3.5))

## plot data
cp = ax.contourf(
    params.cycle, params.time, params["BJ_ac"].mean("member"), cmap="cmo.amp", levels=10
)

## set ticks and add guideline
ax.set_xticks([1, 7, 12], labels=["Jan", "Jul", "Dec"])
ax.axvline(7, c="w", ls="--", lw=1, alpha=0.8)

## add colorbar
cb = fig.colorbar(cp, label=r"Growth rate (yr$^{-1}$)")

## label
ax.set_title("Ensemble-mean growth rate")
ax.set_xlabel("Month")
ax.set_ylabel("Year")

plt.show()

#### Plot annual max growth rate

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3))

## plot ensemble members
for i, m in enumerate(params.member.values):

    label = "Ensemble members" if (i == 50) else None

    ax.plot(
        params.time,
        params["BJ_ac"].sel(member=m).max("cycle"),
        c="gray",
        alpha=0.5,
        lw=0.5,
    )

## plot ensemble mean
ax.plot(
    params.time,
    params["BJ_ac"].mean("member").max("cycle"),
    c="k",
    lw=2,
    label="Ensemble mean",
)

## add labels and set plot style
ax.axhline(0, c="k", ls="--", lw=1)
ax.set_ylim([None, 0.5])
ax.set_xlabel("Time")
ax.set_ylabel(r"Growth rate (yr$^{-1}$)")
ax.set_title("Max annual growth rate")
ax.legend()

plt.show()

## Compare stats b/n RO and MPI (To-do)

### Stochastic integration

In [ ]:
## Fit model on first 30 years of dataset and last 30 years of dataset


def get_RO_ensemble(data, T_var="T_3", h_var="h_w", verbose=False):
    """get RO params for each ensemble member"""

    ## initialize model
    model = XRO(ncycle=12, ac_order=1, is_forward=True)

    ## empty list to hold model fits
    fits = []

    ## Loop thru ensemble members
    for m in tqdm.tqdm(data.member, disable=not (verbose)):

        ## select ensemble member and variables
        data_subset = data[[T_var, h_var]].sel(member=m)

        ## fit model
        with warnings.catch_warnings(action="ignore"):
            fits.append(model.fit_matrix(data_subset, maskNT=[], maskNH=[]))

    return model, xr.concat(fits, dim=data.member)

### Get ensemble of RO models for early/late period

In [ ]:
## get data for early/late period
Th_early = Th.isel(time=slice(None, 600))
Th_late = Th.isel(time=slice(-600, None))

model, RO_params_early = get_RO_ensemble(Th_early, verbose=True)
_, RO_params_late = get_RO_ensemble(Th_late, verbose=True)

### Get ensemble of RO simulations

In [ ]:
seed = 1000
RO_ensemble_early = model.simulate(
    fit_ds=RO_params_early.mean("member"),
    X0_ds=Th[["T_3", "h_w"]].isel(time=0, member=0),
    nyear=50,
    ncopy=100,
    noise_type="red",
    seed=seed,
    is_xi_stdac=False,
    xi_B=0.0,
    is_heaviside=True,
)

#### Seasonal synchronization

In [ ]:
## specify which variable to plot
var_name = "T_3"

## compute std for each dataset
RO_std = RO_ensemble_early.groupby("time.month").std()
mpi_std = Th_early.groupby("time.month").std()

fig, ax = plt.subplots(figsize=(4, 3))

ax.plot(mpi_std.month, mpi_std[var_name].mean("member"))
ax.plot(
    mpi_std.month,
    mpi_std[var_name].mean("member") + mpi_std[var_name].std("member"),
    c="k",
)
ax.plot(mpi_std.month, RO_std[var_name].mean("member"))
ax.plot(mpi_std.month, RO_std[var_name].mean("member") + RO_std[var_name].std("member"))